# Stable Diffusion WebUI 带lora训练

## 阿里云 DSW 一键脚本 By bilibili@十字鱼
### 基于bilibili@秋葉aaaki大佬的云端包修改并整合

#### 简介

- 十字鱼 https://space.bilibili.com/893892
- 更新日期 05.05

## 1 安装 第一次运行时执行

### 1.1 SD安装

In [ ]:
!apt update
!apt install -y aria2
%cd /mnt/workspace
!git clone https://gitcode.net/overbill1683/stable-diffusion-webui
%cd stable-diffusion-webui
!mkdir repositories
%cd repositories
!git clone "https://gitcode.net/overbill1683/stablediffusion" "stable-diffusion-stability-ai"
!git clone "https://gitcode.net/overbill1683/taming-transformers" "taming-transformers"
!git clone "https://gitcode.net/overbill1683/k-diffusion" "k-diffusion"
!git clone "https://gitcode.net/overbill1683/CodeFormer" "CodeFormer"
!git clone "https://gitcode.net/overbill1683/BLIP" "BLIP"
%cd /mnt/workspace/stable-diffusion-webui
!wget -O "config.json" "https://gitcode.net/Akegarasu/sd-webui-configs/-/raw/master/config.json"
# 安装常用插件
extension_urls = """
https://gitcode.net/ranting8323/a1111-sd-webui-tagcomplete
https://gitcode.net/overbill1683/stable-diffusion-webui-localization-zh_Hans
https://gitcode.net/ranting8323/sd-webui-additional-networks
https://gitcode.net/ranting8323/sd-webui-infinite-image-browsing
https://gitcode.net/ranting8323/stable-diffusion-webui-wd14-tagger
"""

extensions = extension_urls.strip().split("\n")

%cd /mnt/workspace/stable-diffusion-webui
for e in extensions:
    !git -C "extensions" clone {e}

### 1.2 lora训练安装
- 不训练可跳过
- ./lora-scripts/sd-scripts文件夹里有文件才成功

In [ ]:
import os
!python -m pip install --upgrade pip
%cd /mnt/workspace
!git clone --recurse-submodules https://ghproxy.com/https://github.com/Akegarasu/lora-scripts
if os.path.exists('lora-scripts'):
    %cd lora-scripts
    !mkdir train
    %cd train
    !mkdir 10_glut
    %cd ../
    !git reset --hard
    !git pull
    !git submodule init
    !git submodule update
    !wget -O "train.sh" "https://ghproxy.com/https://github.com/gluttony-10/lora-scripts/blob/main/train.sh"
    !pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 -f https://mirror.sjtu.edu.cn/pytorch-wheels/torch_stable.html -i https://mirrors.bfsu.edu.cn/pypi/web/simple
    !pip install -U -I --no-deps xformers==0.0.17 -i https://mirrors.bfsu.edu.cn/pypi/web/simple
    if os.path.exists('sd-scripts'):
        %cd sd-scripts
        !pip install --upgrade -r requirements.txt -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !pip install --upgrade lion-pytorch dadaptation -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !pip install --upgrade lycoris-lora -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !pip install --upgrade fastapi uvicorn -i https://mirrors.bfsu.edu.cn/pypi/web/simple
        !sudo apt-get install -y nvidia-cuda-toolkit

## 2 下载模型和VAE

In [ ]:
def aria2(url, filename, d):
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {d}

model_url = "https://huggingface.co/swl-models/chilloutmix/resolve/main/Chilloutmix.safetensors"
aria2(model_url, model_url.split("/")[-1], "/mnt/workspace/stable-diffusion-webui/models/Stable-diffusion")

VAE_URL = "https://huggingface.co/Yukihime256/840000/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt"
aria2(VAE_URL, VAE_URL.split("/")[-1], "/mnt/workspace/stable-diffusion-webui/models/VAE")

## 3 下载其他文件
- 不下载可跳过
- 修改对应文本，运行即可下载文件

In [ ]:
DOWNLOAD_URL = "https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned-fp16.ckpt"
SAVE_DIR = "/mnt/workspace/stable-diffusion-webui/models/Stable-diffusion"
aria2(DOWNLOAD_URL, DOWNLOAD_URL.split("/")[-1], SAVE_DIR)

## 4 训练lora
- 不训练可跳过
- 训练文件放进./lora-scripts/train/10_glut中
- 训练参数请修改 ./lora-scripts/train.sh

In [ ]:
%cd /mnt/workspace/lora-scripts
!bash train.sh

## 5 启动WebUI

In [ ]:
import os
package_envs  = [
      {
        "env": "GFPGAN_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/GFPGAN.git@8d2447a2d918f8eba5a4a01463fd48e45126a379"
      },
      {
        "env": "CLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/CLIP.git@d50d76daa670286dd6cacf3bcd80b5e4823fc8e1"
      },
      {
        "env": "OPENCLIP_PACKAGE",
        "url": "git+https://gitcode.net/overbill1683/open_clip.git@bb6e834e9c70d9c27d0dc3ecedeebeaeb1ffad6b"
      }
]
os.environ["PIP_INDEX_URL"] = "https://mirrors.bfsu.edu.cn/pypi/web/simple"
for i in package_envs:
    os.environ[i["env"]] = i["url"]

%cd /mnt/workspace/stable-diffusion-webui
!python launch.py --no-download-sd-model --xformers --share --listen --enable-insecure-extension-access

## 6 压缩outputs文件夹
- 运行前先停止步骤5

In [ ]:
!tar -zcvf outputs.tar.gz outputs